In [1]:
import astropy.coordinates as coord
from astropy.table import Table, vstack
from astropy.io import ascii 
import astropy.units as u
import healpy
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from astroquery.gaia import Gaia
Gaia.login(user='lander01', password='EsaGaia1234!')

from astropy.io import fits
import os.path

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
q = '''SELECT
  gaia_healpix_index({level}, source_id) AS healpixy,
  count(*) AS num,
  avg(pmra) AS avg_pmra,
  avg(pmdec) AS avg_pmdec
FROM gaiadr1.tgas_source
WHERE (parallax < {plx_max:.2f}) AND (parallax > {plx_min:.2f}) AND (parallax/parallax_error > 5)
GROUP BY healpixy
'''
#halo stars 

haloq = """SELECT *
FROM gaiadr2.gaia_source             
WHERE ABS((1000/parallax)*SIN(RADIANS(b))) > 750 AND MOD(random_index, 10000) = 0"""

#2D dust map

dustq = """SELECT
  gaia_healpix_index(11, source_id) AS healpixy,
  count(*) AS num,
  LOG(SUM(EXP(a_g_val*a_g_val))) AS a_g_sum_sq,
  LOG(AVG(EXP(a_g_val))) AS avg_a_g
FROM gaiadr2.gaia_source
WHERE (parallax < 1) AND (parallax IS NOT NULL) AND (a_g_val IS NOT NULL)
GROUP BY healpixy"""

redHaloCount = """SELECT COUNT(*)
FROM gaiadr2.gaia_source
WHERE (1/parallax)*SIN(RADIANS(b)) > 1 AND (bp_rp - e_bp_min_rp_val > 2.0) AND (bp_rp - e_bp_min_rp_val < 4.0)
"""

redHaloSky = """SELECT *
FROM gaiadr2.gaia_source
WHERE (parallax < 1.) AND (b >= {bmin:.2f}) AND (b < {bmax:.2f})
  AND (bp_rp > 2.0) AND (bp_rp < 5.0)
  AND (l >= {lmin:.2f}) AND (l < {lmax:.2f})"""

blueHaloSky = """SELECT *
FROM gaiadr2.gaia_source
WHERE (parallax < 1.) AND (b >= {bmin:.2f}) AND (b < {bmax:.2f})
  AND (bp_rp > 1.0) AND (bp_rp <= 2.0)
  AND (l >= {lmin:.2f}) AND (l < {lmax:.2f})"""



In [ ]:
ledges = [0, 90, 180, 270, 360]
bmins = [-90, -45, 30, 45]
bmaxs = [-45, -30, 45, 90]

jobnamePre = 'gaiaBlueHalo{bmin}_{bmax}_{lmin}_{lmax}'

for lmin, lmax in zip(ledges[:-1], ledges[1:]):
    for bmin, bmax in zip(bmins, bmaxs):
        jobname = jobnamePre.format(bmin=bmin,bmax=bmax,lmin=lmin,lmax=lmax)
        if os.path.isfile(jobname + '.fits'): continue
        else:
            query = blueHaloSky.format(bmin=bmin, bmax=bmax, lmin=lmin, lmax=lmax)
            try: 
                job = Gaia.launch_job_async(query, name=jobname)
                jobid = job.get_jobid()
            except ConnectionResetError:
                job = Gaia.load_async_job(jobid)
            tbl = job.get_results()
            failed_type = tbl["designation"].dtype
            for name in tbl.dtype.names:
                if tbl[name].dtype == failed_type:
                    tbl[name] = tbl[name].astype(str)
            tbl.write(jobname + '.fits')
            Gaia.remove_jobs([job.get_jobid()])

Launched query: 'SELECT *
FROM gaiadr2.gaia_source
WHERE (parallax < 1.) AND (b >= -90.00) AND (b < -45.00)
  AND (bp_rp > 1.0) AND (bp_rp <= 2.0)
  AND (l >= 0.00) AND (l < 90.00)'
Retrieving async. results...


In [5]:
import glob

In [15]:
files = glob.glob('gaiaRedHalo*.fits')
files

['gaiaRedHalo-45_-30_0_90.fits',
 'gaiaRedHalo-45_-30_180_360.fits',
 'gaiaRedHalo-45_-30_90_180.fits',
 'gaiaRedHalo-90_-45_0_90.fits',
 'gaiaRedHalo-90_-45_180_360.fits',
 'gaiaRedHalo-90_-45_90_180.fits',
 'gaiaRedHalo30_45_0_90.fits',
 'gaiaRedHalo30_45_180_360.fits',
 'gaiaRedHalo30_45_90_180.fits',
 'gaiaRedHalo45_90_0_90.fits',
 'gaiaRedHalo45_90_180_360.fits',
 'gaiaRedHalo45_90_90_180.fits']

In [16]:
hdu = fits.open(files[0])
gaiaData = Table(hdu[1].data)

for f in files[1:]:
    hdu = fits.open(f)
    print(f, len(hdu[1].data))
    gaiaData = vstack((gaiaData, Table(hdu[1].data)))

gaiaRedHalo-45_-30_180_360.fits 928823
gaiaRedHalo-45_-30_90_180.fits 350593
gaiaRedHalo-90_-45_0_90.fits 308880
gaiaRedHalo-90_-45_180_360.fits 555719
gaiaRedHalo-90_-45_90_180.fits 240009
gaiaRedHalo30_45_0_90.fits 779893
gaiaRedHalo30_45_180_360.fits 779893
gaiaRedHalo30_45_90_180.fits 354813
gaiaRedHalo45_90_0_90.fits 309757
gaiaRedHalo45_90_180_360.fits 501933
gaiaRedHalo45_90_90_180.fits 227135


In [11]:
gaiaData

solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,phot_variable_flag,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,datalink_url,epoch_photometry_url
int64,str28,int64,int64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,float32,float32,float64,float64,int16,bool,float32,float64,float64,float32,int16,int16,float32,int32,int16,bool,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,float32,int16,float32,float32,float32,float64,float64,int32,float32,float32,float32,str13,float64,float64,float64,float64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,str77,str99
1635721458409799680,Gaia DR2 6680608701630013184,6680608701630013184,633162250,2015.5,306.49236966972484,0.34895780750456246,-41.053155929907014,0.34751222385784963,0.37276119970039867,0.4526920771517934,0.82343215,12.132489237907265,0.6038034981623494,2.145998506998531,0.652382083335308,0.41386843,0.09302671,-0.07512965,-0.47285938,-0.02942749,-0.33389625,-0.4573146,-0.4052529,-0.17855957,0.3307476,175,0,172,3,1.1303481,187.79951,0.4052938479487371,0.23103999369959838,31,False,0.1432145,1.2636486275923275,0.07957975537015369,0.17061317,20,12,0.69343174,0,21,False,181,340.28547744057215,1.207858360525497,281.72632,19.358757,17,97.10212333119394,6.071009749659444,15.994394,20.383316,20,393.4362932247231,6.802140252717126,57.840073,18.274734,1.4415497,0,2.1085815,1.024559,1.0840225,nan,nan,0,nan,nan,nan,NOT_AVAILABLE,0.047782810121176855,-34.64672830582538,298.74992204884524,-21.186989690509556,999999,nan,nan,nan,nan,nan,nan,nan,nan,nan,999999,nan,nan,nan,nan,nan,nan,http://geadata.esac.esa.int/data-server/datalink/links?ID=6680608701630013184,?
1635721458409799680,Gaia DR2 6680584276152361216,6680584276152361216,624420139,2015.5,306.6472093830671,0.2739827274291423,-41.10553774207437,0.23919924601591094,0.4968586568174306,0.39366823162118847,1.2621254,1.8538001342415995,0.5159329528816636,1.082503879734341,0.484461476097686,0.26649052,0.2493097,-0.09542112,-0.4198123,0.15782827,-0.3171661,-0.54483056,-0.4090531,-0.2499636,0.17086123,253,0,253,0,1.1930772,274.81683,0.0,0.0,31,False,0.17393619,1.4845385652321552,0.06550893038373663,0.03225444,29,12,0.5267055,0,31,True,271,363.6979443239431,1.0096301753505417,360.